In [1]:
import pandas as pd 
import numpy as np
from bertopic import BERTopic 
import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
df = pd.read_excel("/Users/hanna/Scape-Save-DAEN690/careeronestop_data.xlsx")

In [3]:
stop_words = set(stopwords.words('english'))
df['description'] = df['description'].apply(lambda x: str(x).lower())
df['description'] = df['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['description'] = df['description'].apply(lambda x: ''.join([word for word in x if word.isalpha() or word.isspace()]))
df['description'] = df['description'].apply(lambda x: x.strip())


In [4]:

lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))
word_tokens = nltk.word_tokenize(str(df['description']))
removing_stopwords = [word for word in word_tokens if word not in stop_words]
lemmatized_word = [lemmatizer.lemmatize(word) for word in removing_stopwords]

In [5]:
# Tokenize the job descriptions
#df['tokens'] = df['description'].apply(lambda x: word_tokenize(x))

#df['lemmatized'] = word_tokens.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


In [13]:
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(df['description']) #when fit lemmatized_word, the topics do not make sense at all, could potentially be the spanish words that is throwing it off 

Batches: 100%|██████████| 129/129 [07:39<00:00,  3.56s/it]
2023-03-10 13:25:34,284 - BERTopic - Transformed documents to Embeddings
2023-03-10 13:26:06,324 - BERTopic - Reduced dimensionality
2023-03-10 13:26:09,161 - BERTopic - Clustered reduced embeddings


In [16]:
freq = topic_model.get_topic_info(); freq.head(5)
topic_model.get_topic(0)  # Select the most frequent topic

[('nan', 4.715753260784361),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05)]

In [15]:
topic_model.visualize_barchart(top_n_topics=5)

In [17]:
freq

,Topic,Count,Name
0,-1,808,-1_de_la_el_en
1,0,254,0_nan___
2,1,183,1_cbre_directory_position_interest
3,2,81,2_penetration_testing_security_team
4,3,78,3_pwc_dirigir_en_nis
...,...,...,...
88,87,11,87_product_devices_rd_cybersecurity
89,88,11,88_cybersecurity_program_dod_system
90,89,11,89_plataforma_arquitectura_la_de
91,90,11,90_eurofins_laboratory_samples_sample
